In [1]:
from pyspark import SparkConf, SparkContext
sc = SparkContext(conf=SparkConf().setAppName("MyApp").setMaster("local"))

In [2]:
import re

def parse_article(line):
    try:
        article_id, text = unicode(line.rstrip()).split('\t', 1)
        text = re.sub("^\W+|\W+$", "", text.lower(), flags=re.UNICODE)
        words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
        return words
    except ValueError as e:
        return []


In [3]:
wiki = sc.textFile("/data/wiki/en_articles_part/articles-part", 16).map(parse_article)
result = wiki.take(1)[0]

In [7]:
print type(wiki), type(result)

<class 'pyspark.rdd.PipelinedRDD'> <type 'list'>


In [12]:
wiki.count()

4100

In [5]:
len(result)

10309

In [6]:
for word in result[:5]:
    print word

anarchism
anarchism
is
often
defined


In [8]:
sc.uiWebUrl

u'http://172.172.1.2:4040'

In [28]:
# define broadcast var with first word in pairs
broadcastVars = sc.broadcast({"word": "narodnaya"})
print type (broadcastVars.value["word"])
broadcastVars.value["word"] in " is narodnaya "

<type 'str'>


True

In [48]:
# filter words when reading file
def wordFilter(line):
    text = line.lower()
    if broadcastVars.value["word"] in text:
        return text
    return ""

In [53]:
def notEmpty(v):
    if len(v) == 0:
        return False
    return True

In [54]:
broadcastVars = sc.broadcast({"word": "narodnaya"})
fwiki = sc.textFile("/data/wiki/en_articles_part/articles-part").map(wordFilter).filter(notEmpty)
fwiki.count()

4

In [58]:
ff = fwiki.take(fwiki.count())
len([f for f in ff if f != ""])

4

In [63]:
# filter words when reading file
def wordFilter2(line):
    text = line.lower()
    if broadcastVars.value["word"] in text:
        return text

In [68]:
broadcastVars = sc.broadcast({"word": "narodnaya"})
fwiki = sc.textFile("/data/wiki/en_articles_part/articles-part").map(wordFilter2).filter(lambda v: v != None )
fwiki.count()

4

In [106]:
# make func that finds target word and emits docs
# 
def extractPairs(line):
    try:
        print "line:", line
        #article_id, text = unicode(line.strip()).split('\t', 1)
        article_id, text = unicode(line.strip()).split('%', 1)
        print "text:", text

        text = re.sub("^\W+|\W+$", "", text.lower(), flags=re.UNICODE)
        words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
        pair = False
        pairs = []
        for word in words:
            print word
            #if broadcastVars.value["word"] == word:
            if broadcastVars.value["word"] == word:
                pair = True
                continue
            if pair:
                pairs.append([broadcastVars.value["word"] + "_" + word,1])
                pair = False
        return pairs
    except ValueError as e:
        print e
        return []


In [107]:
s = "111 %   Since you already have narodnaya mapper's and reducer's code you can emulate Hadoop's behavior using a simple bash:1Since you already have mapper's and narodnaya reducer's code"
extractPairs(s)

need more than 1 value to unpack


[]

In [81]:
print broadcastVars.value["word"] == "narodnaya"

True


In [159]:
def extractPairs2(line):
    try:
        #print "line:", line
        article_id, text = unicode(line.strip()).split('\t', 1)
        #article_id, text = unicode(line.strip()).split('%', 1)
        #print "text:", text

        text = re.sub("^\W+|\W+$", "", text.lower(), flags=re.UNICODE)
        words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
        pair = False
        pairs = []
        for word in words:
            #print word
            #if broadcastVars.value["word"] == word:
            if broadcastVars.value["word"] == word:
                pair = True
                continue
            if pair:
                pairs.append(broadcastVars.value["word"] + "_" + word)
                pair = False
        #print "return pairs", pairs
        return pairs
    except ValueError as e:
        print e
        return []


In [160]:
keyWord = fwiki.flatMap(extractPairs2)
keyWord.collect()

[u'narodnaya_volya',
 u'narodnaya_volya',
 u'narodnaya_volya',
 u'narodnaya_volya',
 u'narodnaya_volya',
 u'narodnaya_volya',
 u'narodnaya_volya',
 u'narodnaya_volya',
 u'narodnaya_volya',
 u'narodnaya_gazeta']

In [163]:
keyWordCount = keyWord.map(lambda k: (k,1))
keyWordCount.take(3)

[(u'narodnaya_volya', 1), (u'narodnaya_volya', 1), (u'narodnaya_volya', 1)]

In [164]:
keyWordCount.reduceByKey(lambda x,y: x+y).collect()

[(u'narodnaya_gazeta', 1), (u'narodnaya_volya', 9)]

In [165]:
res = keyWordCount.reduceByKey(lambda x,y: x+y).collect()

In [166]:
for pair in res:
    print pair

(u'narodnaya_gazeta', 1)
(u'narodnaya_volya', 9)


In [150]:
# word count
text_file = sc.textFile("README.md")
counts = text_file.flatMap(lambda line: line.split(" ")) \
             .map(lambda word: (word, 1)) \
             .reduceByKey(lambda a, b: a + b)
counts.take(10)
#counts.saveAsTextFile("hdfs://...")

[(u'', 22),
 (u'text', 1),
 (u'results', 1),
 (u'symbols', 3),
 (u'grades', 1),
 (u'"404"', 1),
 (u'corner', 1),
 (u'submit', 1),
 (u'clicked', 1),
 (u'cycle.', 1)]

In [151]:
counts.keys().take(5)

[u'', u'text', u'results', u'symbols', u'grades']

In [146]:
!ls

4week-examples.ipynb  DemoNotebook.ipynb  README.md  WordPairs.ipynb


In [158]:
text_file.map(lambda line: line.split(" ")).take(3)

[[u'##', u'How', u'to', u'use', u'LTI', u'interface'],
 [u''],
 [u'While',
  u'completing',
  u'the',
  u'programming',
  u'assignment,',
  u'you',
  u'have',
  u'to',
  u'submit',
  u'your',
  u'assignment.',
  u'In',
  u'order',
  u'to',
  u'make',
  u'submission,',
  u'you',
  u'have',
  u'to',
  u'create',
  u'a',
  u'notebook.',
  u'When',
  u'you',
  u'created',
  u'a',
  u'Jupyter',
  u'notebook,',
  u'in',
  u'the',
  u'right',
  u'top',
  u'corner',
  u'you',
  u'could',
  u'see',
  u'a',
  u'dropdown',
  u'with',
  u'tasks',
  u'and',
  u'a',
  u'button',
  u'"Submit',
  u'assignment".',
  u'You',
  u'have',
  u'to',
  u'choose',
  u'the',
  u'appropriate',
  u'task',
  u'(the',
  u'task',
  u'that',
  u'you',
  u'have',
  u'chosen',
  u'from',
  u'Coursera',
  u'Interface)',
  u'and',
  u'press',
  u'the',
  u'blue',
  u'button.',
  u'']]

In [155]:
text_file.flatMap(lambda line: line.split(" ")).take(3)

[u'##', u'How', u'to']

In [156]:
text_file.flatMap(lambda line: line.split(" ")).map(lambda word: (word, 1)).take(3)

[(u'##', 1), (u'How', 1), (u'to', 1)]

In [157]:
text_file.flatMap(lambda line: line.split(" ")).map(lambda word: (word, 1)).reduceByKey(lambda a,b: a+b).take(3)

[(u'', 22), (u'text', 1), (u'results', 1)]